In [ ]:
from pyclustering.cluster.bsas import bsas
import numpy as np
import matplotlib.pyplot as plt
import cv2
fig_size =[12,9]
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
img = cv2.imread('faulty_image.jpg')
img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,img_thresh = cv2.threshold(img_gray,100,255,cv2.THRESH_TOZERO)
nonzro_samples = cv2.findNonZero(img_thresh).reshape(-1, 2).astype('float32')
plt.imshow(img_thresh,cmap='gray')
plt.show()

In [ ]:
max_clusters = 8
threshold = 20
bsas_instance = bsas(nonzro_samples, max_clusters, threshold)
bsas_instance.process()
clusters = bsas_instance.get_clusters()
#representatives = bsas_instance.get_representatives()

In [ ]:
cms=[]
ROIs=np.zeros((len(clusters),4))
for i,cluster in enumerate(clusters):
    current_batch=nonzro_samples[cluster]
    cms.append(np.sum(current_batch,axis=0)/current_batch.shape[0])
    row_max=np.max(current_batch[:,1],axis=0)+6
    row_min=np.min(current_batch[:,1],axis=0)-6
    col_max=np.max(current_batch[:,0],axis=0)+6
    col_min=np.min(current_batch[:,0],axis=0)-6
    ROIs[i,:]=[row_min,row_max,col_min,col_max]

In [ ]:
image_ROIs=[]
for roi in ROIs.astype('int32'):
    print(roi)
    image_ROIs.append(img_thresh.copy()[roi[0]:roi[1],roi[2]:roi[3]])
plt.imshow(image_ROIs[0],cmap='gray')
plt.show()

In [ ]:
index=0 #Which ROI do you want to consider?
frame=image_ROIs[index]
roi_range=ROIs[index].astype('float32')
params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 50;
params.maxThreshold = 255;
params.filterByArea = True
params.minArea = 0
params.filterByCircularity = True
params.minCircularity = 0.1
params.filterByConvexity = True
params.minConvexity = 0.1
params.filterByInertia = True
params.minInertiaRatio = 0.1
params.blobColor = 255
 
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3 :
    detector = cv2.SimpleBlobDetector(params)
else : 
    detector = cv2.SimpleBlobDetector_create(params)

keypoints = detector.detect(frame)
print(keypoints[0].pt)
print(roi_range)

In [ ]:
frame_clr=cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB)
im_with_keypoints = cv2.drawKeypoints(frame_clr, keypoints, np.array([]), (255,0,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(frame_clr)
for key in keypoints:
    plt.plot(key.pt[0],key.pt[1],'r*')
plt.show()
img = cv2.imread('faulty_image.jpg')
# roi_range[2] corresponds to roi_x_min and keypoints[0].pt[0] corresponds to the x component of the keypoint
for key in keypoints:
    cv2.circle(img,(int(round(key.pt[0]+roi_range[2])), int(round(key.pt[1]+roi_range[0]))), 2, (255,0,255), -1)
plt.imshow(img)
plt.show()

In [ ]:
len(keypoints)

In [1]:
import cv2
import numpy as np
from pyclustering.cluster.bsas import bsas
class markerExteractor(object):
    def __init__(self):
        self.max_clusters = 8
        self.threshold = 20
        self.blubParams = cv2.SimpleBlobDetector_Params()
        self.blubParams.minThreshold = 50;
        self.blubParams.maxThreshold = 255;
        self.blubParams.filterByArea = True
        self.blubParams.minArea = 0
        self.blubParams.filterByCircularity = True
        self.blubParams.minCircularity = 0.3
        self.blubParams.filterByConvexity = True
        self.blubParams.minConvexity = 0.7
        self.blubParams.filterByInertia = True
        self.blubParams.minInertiaRatio = 0.1
        self.blubParams.blobColor = 255
        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
            self.blubDetector = cv2.SimpleBlobDetector(self.blubParams)
        else : 
            self.blubDetector = cv2.SimpleBlobDetector_create(self.blubParams)
    def detect(self,frame):
        self.cms=[]
        self.image_ROIs=[]
        self.keypoints=[]
        img_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret,img_thresh = cv2.threshold(img_gray,100,255,cv2.THRESH_TOZERO)
        #Find the clusters
        self.nonzro_samples = cv2.findNonZero(img_thresh)
        if self.nonzro_samples is None:
            return None
        else:
            self.nonzro_samples=self.nonzro_samples.reshape(-1, 2).astype('float32')
        bsas_instance = bsas(self.nonzro_samples, self.max_clusters, self.threshold)
        bsas_instance.process()
        clusters = bsas_instance.get_clusters()
        #Calculate the center of the clusters and the Regions of Interests
        self.ROIs=np.zeros((len(clusters),4))
        for i,cluster in enumerate(clusters):
            current_batch=self.nonzro_samples[cluster]
            self.cms.append(np.sum(current_batch,axis=0)/current_batch.shape[0])
            row_max=np.max(current_batch[:,1],axis=0)+6
            row_min=np.min(current_batch[:,1],axis=0)-6
            col_max=np.max(current_batch[:,0],axis=0)+6
            col_min=np.min(current_batch[:,0],axis=0)-6
            self.ROIs[i,:]=[row_min,row_max,col_min,col_max]
        for roi in self.ROIs.astype('int32'):
            self.image_ROIs.append(img_thresh.copy()[roi[0]:roi[1],roi[2]:roi[3]])
        #Return The Results
        marker_points=[]
        for i,roi in enumerate(self.image_ROIs):
            keys_in_roi=self.blubDetector.detect(roi)
            for key in keys_in_roi:
                #Calculate the global coordinate of marker points. The points are returned in (X(Col),Y(Row)) coordinate. 
                marker_points.append([key.pt[0]+self.ROIs.astype('float32')[i,2],key.pt[1]+self.ROIs.astype('float32')[i,0]])
        return np.array(marker_points)

In [ ]:
img = cv2.imread('faulty_image.jpg')
markerExteractor_inst=markerExteractor()
points=markerExteractor_inst.detect(img)
for i in range(len(points)):
    cv2.circle(img,(int(round(points[i,0])), int(round(points[i,1]))), 2, (255,0,255), -1)
plt.imshow(img)
plt.show()
cv2.imwrite('res.png',img)

In [ ]:
#Testing the algorithm with a stream of images
markerExteractor_inst=markerExteractor()
cap=cv2.VideoCapture('/home/rouholla/Stereo_6DOF_Tracker/output_rigt.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_debug.avi', fourcc, 30.0, (int(cap.get(3)),int(cap.get(4))))
while cap.isOpened():
    ret,img=cap.read()
    raw_img=img.copy()
    if ret==True:
        points=markerExteractor_inst.detect(img)
        if points is not None:
            for i in range(len(points)):
                cv2.circle(img,(int(round(points[i,0])), int(round(points[i,1]))), 2, (255,0,255), -1)
        cv2.imshow('Frame',img)
        out.write(img)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            cv2.imwrite('faulty_image.jpg',raw_img)
            break
    else:
        break #Break the while loop if no frames could be captured
cv2.destroyAllWindows()     
out.release()
cap.release()

In [191]:
import yaml
class undistrodMarkers:
    def __init__(self,config_file_name):
        with open(config_file_name, 'r') as f:
            calib = yaml.safe_load(f.read())
        self.K = np.array(calib['camera_matrix']['data']).reshape(calib['camera_matrix']['rows'],calib['camera_matrix']['cols'])
        self.D = np.array(calib['distortion_coefficients']['data']).reshape(-1, 5)
        self.P = np.array(calib['projection_matrix']['data']).reshape(3, 4)
        self.R = np.array(calib['rectification_matrix']['data']).reshape(3, 3)
        self.img_width = calib['image_width']
        self.img_height = calib['image_height']
    def process(self,points):
        lpts_ud=cv2.undistortPoints(points.reshape(-1,1,2).astype(np.float32), self.K, self.D,P=self.P,R=self.R)
        return cv2.convertPointsToHomogeneous(np.float32(lpts_ud))

leftUndist = undistrodMarkers('/home/rouholla/Stereo_6DOF_Tracker/left.yaml')
rightUndist = undistrodMarkers('/home/rouholla/Stereo_6DOF_Tracker/right.yaml')
        

## Phase 2
Now we use the tracker we just created to build our 6DOF tracker system

In [4]:
#Testing the algorithm with a stream of images
markerExteractor_inst=markerExteractor()
cap_right=cv2.VideoCapture('/home/rouholla/Stereo_6DOF_Tracker/ouput_right.avi')
cap_left=cv2.VideoCapture('/home/rouholla/Stereo_6DOF_Tracker/output_right.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_debug.avi', fourcc, 30.0, (int(cap_left.get(3)),int(cap_left.get(4))))

while cap_left.isOpened():
    ret,img_left=cap_left.read()
    ret,img_right=cap_right.read()
    if ret==True:
        points_left=markerExteractor_inst.detect(img_left)
        points_right=markerExteractor_inst.detect(img_right)
        if (points_left is not None) and (points_right is not None):
            left_ud=leftUndist.process(points_left)
            right_ud=rightUndist.process(points_right)
            for i in range(min(len(points_left),len(points_right))):
                cv2.circle(img_left,(int(round(points_left[i,0])), int(round(points_left[i,1]))), 2, (255,0,255), -1)
                cv2.circle(img_right,(int(round(points_right[i,0])), int(round(points_right[i,1]))), 2, (255,0,255), -1)

                
        cv2.imshow('Frame',np.hstack([img_left,img_right]))
        out.write(img_left)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            cv2.imwrite('faulty_image.jpg',img_right)
            break
    else:
        break #Break the while loop if no frames could be captured
cv2.destroyAllWindows()     
out.release()
cap_left.release()
cap_right.release()

In [ ]:
cv2.destroyAllWindows()     


In [ ]:
#lpts=np.array([329.875, 378.875]).reshape(1,2)
#rpts=np.array([336.5, 334.5]).reshape(1,2)
#left_ud=leftUndist.process(lpts)
#right_ud=rightUndist.process(rpts)
print(left_ud.reshape(-1,3))
print ('right')
print(right_ud.reshape(-1,3))

In [ ]:
left_ud-right_ud
left_ud.reshape(-1,3)[:,0:2]

In [312]:
#Create the corresponding matrix
def calculate_connection_mateix(left_points,right_points):
    M=len(left_points)
    N=len(right_points)
    connection_matrix=np.zeros((M,N))
    for i in range(0,M):
        for j in range(0,N):
                if(np.abs(left_points[i,1]-right_points[j,1])<10):
                       connection_matrix[i,j]=1
    return  connection_matrix

def get_secondary_correspondings(matchesA,matchesB,points_left,points_right):
    if len(matchesA)==0 or len(matchesB)==0:
        return np.array([]),np.array([])
    points_in_matchesA=[]
    points_in_matchesB=[]
    for clusters in matchesA:
        points_in_cluster=points_left[clusters,:]
        points_in_matchesA.append(points_in_cluster[points_in_cluster[:,0].argsort(),:])
    for clusters in matchesB:
        points_in_cluster=points_right[clusters,:]
        points_in_matchesB.append(points_in_cluster[points_in_cluster[:,0].argsort(),:])

    points_in_matchesA=np.vstack(points_in_matchesA)
    points_in_matchesB=np.vstack(points_in_matchesB)
    if points_in_matchesA.shape[0]==points_in_matchesB.shape[0]:
        return points_in_matchesA,points_in_matchesB
    else:
        L=points_in_matchesA
        R=points_in_matchesB
        if(L.shape[0]<R.shape[0]):
            dists=np.ma.array([(R-L[i,:])[:,1] for i in range(L.shape[0])],mask=False)
            chosens=[]
            for i in range(dists.shape[0]):
                dists.mask[:,chosens]=True
                chosens.append(np.argmin(dists[i,:]))
            R=R[chosens,:]
        else:

            dists=np.ma.array([(L-R[i,:])[:,1] for i in range(R.shape[0])],mask=False).reshape(R.shape[0],-1)
            chosens=[]
            for i in range(dists.shape[0]):
                dists.mask[:,chosens]=True
                #print('a')
                #print(dists)
                chosens.append(np.argmin(dists[i,:]))
            L=L[chosens,:]
            
        return L,R


def process_filtered_connection_matrix(connection_matrix):
    m,n=connection_matrix.shape
    matchesA=[]
    matchesB=[]
    for i in range(m):
        if connection_matrix[i,:].any()==1:
            matche=np.where((connection_matrix==connection_matrix[i,:]).all(axis=1))[0] #What rows are identical
            if matche.tolist() not in matchesA:# if it's a new kind of row save the place where they exist
                matchesA.append(matche.tolist())
                matchesB.append(np.where(connection_matrix[i,:]==1)[0].tolist())
    return matchesA,matchesB 
def filter_connection_matrix(connection_matrix,left_points,right_points):
    new_connection_matrix=connection_matrix.copy()
    progress=1
    old_cost=np.sum(connection_matrix)
    correspondings=[]
    while progress>0:
        m,n=connection_matrix.shape
        for i in range(m):
            if np.sum(connection_matrix[i,:],axis=0)==1: #left marker i is connected to just one right marker j
                j=np.where(connection_matrix[i,:]==1)
                connection_matrix[:,j]=0
                new_connection_matrix[:,j]=0
                connection_matrix[i,j]=1
                if (i,j[0][0]) not in correspondings:
                    correspondings.append([i,j[0][0]])
        for i in range(n):
            if np.sum(connection_matrix[:,i],axis=0)==1: #left marker i in the right is connected to just one the left 
                j=np.where(connection_matrix[:,i]==1)
                connection_matrix[j,:]=0
                new_connection_matrix[j,:]=0
                connection_matrix[j,i]=1
                if [j[0][0],i] not in correspondings:
                    correspondings.append([j[0][0],i])
        progress=old_cost-np.sum(connection_matrix)
        old_cost=np.sum(connection_matrix)
        correspondings=np.array(correspondings)
        if correspondings.size != 0:
               return new_connection_matrix,left_points[correspondings[:,0],:],right_points[correspondings[:,1],:]
        else:
               return new_connection_matrix,np.array([]),np.array([])
def get_correspondings(left_points,right_points):
    connection_matrix=calculate_connection_mateix(left_points,right_points)
    new_cm,immediate_correspondings_left,immediate_correspondings_right=filter_connection_matrix(connection_matrix,left_points,right_points)
    matchesLeft,matchesRight=process_filtered_connection_matrix(new_cm)
    left_corr_sec,right_corr_sec=get_secondary_correspondings(matchesLeft,matchesRight,left_points,right_points)
    if immediate_correspondings_left.size !=0 and left_corr_sec.size!=0:
        return np.vstack([left_corr_sec,immediate_correspondings_left]),np.vstack([right_corr_sec,immediate_correspondings_right])
    elif immediate_correspondings_left.size ==0 and left_corr_sec.size!=0:
        return np.vstack([left_corr_sec]),np.vstack([right_corr_sec])
    else:
        return np.vstack([immediate_correspondings_left]),np.vstack([immediate_correspondings_right])


In [202]:
#Test the corrosponding finder algorithm using dummy data
#left_points=np.array([[104,110],[201,110],[200,100],[100,100],[202,120],[102,120],[10,100]]).reshape(-1,2)
#right_points=np.array([[101,200],[202,220],[102,220],[103,210],[201,200],[203,210],[12,120]]).reshape(-1,2)
left_points=np.array([[104,110],[201,110],[200,100]]).reshape(-1,2)
right_points=np.array([[101,200],[202,220],[102,220],[103,210],[201,200],[203,210],[12,120]]).reshape(-1,2)
left_points[:,[0,1]]=left_points[:,[1,0]]
right_points[:,[0,1]]=right_points[:,[1,0]]
cm=np.array([[1,0,1,1,0,0],[0,1,0,0,1,1],[1,0,1,1,0,0],[0,1,0,0,1,1],[0,1,0,0,1,1],[1,0,1,1,0,0]])

connection_matrix=calculate_connection_mateix(left_points,right_points)
new_cm,immediate_correspondings_left,immediate_correspondings_right=filter_connection_matrix(connection_matrix,left_points,right_points)
matchesLeft,matchesRight=process_filtered_connection_matrix(new_cm)
#L,R=get_secondary_correspondings(matchesLeft,matchesRight,left_points,right_points)
left_corr,right_corr=get_correspondings(left_points,right_points)

#A is the Matrix with less points
left_corr,right_corr

(array([[100, 200],
        [110, 201],
        [110, 104]]), array([[200, 201],
        [220, 202],
        [200, 101]]))

In [203]:
#get the corrosponding points for the exteracted points from the camera frames
lpts_ud,rpts_ud=get_correspondings(left_ud.reshape(-1,3)[:,0:2],right_ud.reshape(-1,3)[:,0:2])
res=cv2.triangulatePoints(leftUndist.P,rightUndist.P,lpts_ud.reshape(-1,1,2).astype(np.float32),rpts_ud.reshape(-1,1,2).astype(np.float32))
points_3d=(res/res[-1,:])[0:3,:]

[]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
np.where((cm==cm[0,:]).all(axis=1))[0].tolist()

In [ ]:
a=[[1,2,3],[4,5,6]]
b=[1,2,3]
b in a

In [ ]:
points_3d


## Les's test the Multi marker 3D point Tracker

In [315]:
#Testing the algorithm with a stream of images
markerExteractor_inst=markerExteractor()
cap_right=cv2.VideoCapture('/home/rouholla/Stereo_6DOF_Tracker/output_right.avi')
cap_left=cv2.VideoCapture('/home/rouholla/Stereo_6DOF_Tracker/output_left.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_debug.avi', fourcc, 30.0, (int(cap_left.get(3)),int(cap_left.get(4))))
markerExteractor_inst.max_clusters = 8
markerExteractor_inst.threshold = 200
markerExteractor_inst.blubParams = cv2.SimpleBlobDetector_Params()
markerExteractor_inst.blubParams.minThreshold = 50;
markerExteractor_inst.blubParams.maxThreshold = 255;
markerExteractor_inst.blubParams.filterByArea = True
markerExteractor_inst.blubParams.minArea = 0
markerExteractor_inst.blubParams.filterByCircularity = True
markerExteractor_inst.blubParams.minCircularity = 0.3
markerExteractor_inst.blubParams.filterByConvexity = True
markerExteractor_inst.blubParams.minConvexity = 0.9
markerExteractor_inst.blubParams.filterByInertia = True
markerExteractor_inst.blubParams.minInertiaRatio = 0.1
markerExteractor_inst.blubParams.blobColor = 255
while cap_left.isOpened():
    ret,img_left=cap_left.read()
    ret,img_right=cap_right.read()
    if ret==True:
        points_left=markerExteractor_inst.detect(img_left)
        points_right=markerExteractor_inst.detect(img_right)
        if (points_left is not None) and (points_right is not None):
            left_ud=leftUndist.process(points_left)
            right_ud=rightUndist.process(points_right)
            for i in range(min(len(points_left),len(points_right))):
                cv2.circle(img_left,(int(round(points_left[i,0])), int(round(points_left[i,1]))), 2, (255,0,255), -1)
                cv2.circle(img_right,(int(round(points_right[i,0])), int(round(points_right[i,1]))), 2, (255,0,255), -1)
        lpts_ud,rpts_ud=get_correspondings(left_ud.reshape(-1,3)[:,0:2],right_ud.reshape(-1,3)[:,0:2])
        
        #print((lpts_ud.shape[0],rpts_ud.shape[0]))
        res=cv2.triangulatePoints(leftUndist.P,rightUndist.P,lpts_ud.reshape(-1,1,2).astype(np.float32),rpts_ud.reshape(-1,1,2).astype(np.float32))
        points_3d=res[0:3,:]/res[-1,:]
        #print(points_3d[-1,-1])
        if points_3d[-1,-1] >1000:
            cv2.imwrite('/home/rouholla/faulty_right_image.jpg',img_right)
            cv2.imwrite('/home/rouholla/faulty_left_image.jpg',img_left)
            print(points_left)
            print(points_right)
            
            break

        cv2.imshow('Frame',np.hstack([img_left,img_right]))
        out.write(img_left)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            cv2.imwrite('faulty_image.jpg',img_right)
            break
    else:
        break #Break the while loop if no frames could be captured
cv2.destroyAllWindows()     
out.release()
cap_left.release()
cap_right.release()

In [294]:
print(points_3d)

[[0.1842181  0.2366365  0.19035164 0.10936636]
 [0.00180847 0.00303939 0.07572664 0.01582983]
 [0.72752655 0.7797519  0.7661044  0.77551484]]


In [ ]:
[np.abs(left_points[i,1]-right_points[j,1]) for i,j in range(3,3)]

In [ ]:
np.abs(left_points[0,1]-right_points[0,1])<10

In [14]:
cv2.destroyAllWindows()